In [ ]:
import numpy as np
import sys
import os
import argparse

In [ ]:
'''
matrix transformation -- 


It defines classes_and_methods

@author:     Jerome

@copyright:  2018 All rights reserved.

@license:    license

@contact:    email
@deffield    
'''
import readline
import sys
import os
import numpy as np
import pandas as pd
from sys import argv



# function of creating connection
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
        sys.exit(1)
    return None

def run():
    # argument reading
    # index of starting task

    # ped file
    # ped_file = args.ped_file
    ped_file = "/ysm-gpfs/home/zy92/project/adversarial_autoencoders/ukb_data/UK_mini_v1.ped"
    
    # map file
    # map_file = args.map_file
    map_file = "/ysm-gpfs/home/zy92/project/adversarial_autoencoders/ukb_data/UK.map"
    
    
    # output file 
    # output_file = args.output_file
    output_file = "/ysm-gpfs/home/zy92/project/adversarial_autoencoders/ukb_data/output_matrix.txt"
    
    
            
    # calculation        
    for k in range(P):
        logging.info("Gene: " + str(k + nstart))
        gene = gene_ensg[k + nstart -1]
        print(gene)
        #read snp list
        #snp_rsid
        
        try:
            filename = cov_dir + "/../snplist/"+ gene + ".snplist"
            snp_rsid = pd.read_table(filename, header = None)
        except:
            continue
        snp_rsid = list(snp_rsid.loc[:,0])
        
        #matrix of weights
        M = len(snp_rsid) #number of snps
        logging.info("Number of SNPs: " + str(M))
        weights = np.zeros(shape = (M, N))
        for i in range(N):
            logging.info("Database: " + str(M))
            dbname = fi[i]
            conn = create_connection(dbname)
            cur = conn.cursor()  
            sql_q = 'select * from weights where gene = "' + gene + '"'
            tmp_query = cur.execute(sql_q).fetchall()
            rsid_in_db = list(map(lambda x: str(x[0]), tmp_query))
            #rsid_in_db = map(lambda x: str(x[0]), tmp_query)
            index = pd.match(rsid_in_db, snp_rsid)
            indi = index[index > -1]
            # extract the weight
            sql_q = 'select * from weights where gene = "' + gene + '"'
            tmp_query = cur.execute(sql_q).fetchall()
            tmp_weights = np.array(list(map(lambda x: str(x[2]), tmp_query)))
            #tmp_weights = np.array(map(lambda x: str(x[2]), tmp_query))
            #print(tmp_weights[index > -1])
            weights[indi,i] = tmp_weights[index > -1]
            
        # covariance matrix of snps
        cov_file = cov_dir + gene_id[k + nstart - 1] + ".cov"
        cov_matrix = np.loadtxt(cov_file)
        logging.info("Cov Matrix")
        print(cov_matrix)
        logging.info("Weights")
        print(weights)
        # covariance matrix of gene in different tissue
        cov_gene = np.mat(weights.T) * np.mat(cov_matrix) * np.mat(weights)
        cov_gene = np.array(cov_gene)
        print(cov_gene)
        # normalization
        for i in range(N):
            if cov_gene[i,i] != 0:
                cov_gene[i,:] = cov_gene[i,:] / np.sqrt(cov_gene[i,i])
                cov_gene[:,i] = cov_gene[:,i] / cov_gene[i,i]
        
        #z-score of gene in different tissue
        zscore_gene = np.full([N, 1], np.nan)   
        for i in range(N):
            nam = "zscore_" + str(i+1)
            index = zscore_dict[nam]["gene"] == gene
            #===================================================================
            # print("Sum of index\n")
            # print(sum(index))
            #===================================================================
            if sum(index) > 0:
                zscore_gene[i] = zscore_dict[nam]["zscore"][index].values[0]
                print("Zscore_index\n")
                print(zscore_gene[i])
                #p-value
                outcome.loc[k, (i+4)] = float(zscore_dict[nam]["pvalue"][index].values[0])
                print("P_value_index\n")
                print(outcome.loc[k, (i+4)])
                print(zscore_dict[nam]["pvalue"][index])
            
        
        #only keep tissues with prediction model for gene
        index = np.isnan(zscore_gene) == False
        indext = index.T[0]
        if sum(index) > 0:
            zscore_gene = zscore_gene[index]
            #===================================================================
            # print("Dim of zscore_gene" + str(len(zscore_gene)))
            # print("Zscore_gene:\n")
            # print(zscore_gene)
            #===================================================================
            cov_gene = cov_gene[indext,:][:,indext]
            #===================================================================
            # print("Dim of cov_gene:")
            # print(cov_gene.shape)
            # print("Cov_gene:\n")
            # print(cov_gene)
            #===================================================================
        else:
            continue
        # check if the matrix is symmetric
        if np.allclose(cov_gene,cov_gene.T):
            # GBJ
            
            r_zscore_gene = r.matrix(zscore_gene)
            ##print(zscore_gene.shape)
            ##print(cov_gene.shape)
            r_cov_gene = r.matrix(cov_gene, nrow = cov_gene.shape[0])
            #print(r_zscore_gene)
            #print(r_cov_gene)
            
            # run the test
            
            GBJ_res = r["GBJ"](test_stats=r_zscore_gene, cor_mat=r_cov_gene)
            
            outcome.loc[k, 2] = GBJ_res.rx2("GBJ")[0]
            print(GBJ_res.rx2("GBJ")[0])
            outcome.loc[k, 3] = GBJ_res.rx2("GBJ_pvalue")[0]
            print(GBJ_res.rx2("GBJ_pvalue")[0])
    # output the results
    os.chdir(out_dir)
    filename = "outcome_" + str(nstart) + "_" + str(nend) + ".txt"
    outcome.to_csv(filename, header=None, index=None, sep='\t', mode='w')
    #with open(filename,"w") as fo:
        #np.savetxt(fo, outcome)
    

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description='Matrix Transformation')

    parser.add_argument("--verbosity",
                        help="Log verbosity level. 1 is everything being logged. 10 is only high level messages, above 10 will hardly log anything",
                        default = "10")

    parser.add_argument("--ped_file",
                        help="path to .ped file",
                        default="./ukb_data/UK_mini_v1.ped")

    parser.add_argument("--map_file",
                        help="path to .map file",
                        default="./ukb_data/UK.map")
    
    parser.add_argument("--output_file",
                        help="path to output file",
                        default="./output_matrix.txt")

   

    args = parser.parse_args()
    
    # run the main function
    run()